# Ejercicio 2 - Parcial

Considerar los datasets de la distancia anual entre Urano y Saturno (en unidades astronómicas)
y la generación de energía nuclear por hora en Brasil (kWh), encontrados en este sitio: https:
//tinyurl.com/2k3h8bz4 (los datos están a mitad de página).

### Ayuda

Si se usan funciones de librería scipy.stats, prestar atención a vectorized=False, paired=True

## a)
Encontrar el estimador plug-in del coeficiente de correlación entre ambos conjuntos de datos.

### Desarrollo

Primero empiezo por leer los datos:

In [1]:
import pandas as pd 
import os 

ruta_datos = os.path.join('..', '..', '..', 'datos', 'raw', 'planetary_distance_nuclear_power.csv')

df_distancia_nuclear = pd.read_csv(ruta_datos)

df_distancia_nuclear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Planetary distance (AU)                           40 non-null     float64
 1   Nuclear power generation in Brazil (Billion kWh)  40 non-null     float64
dtypes: float64(2)
memory usage: 772.0 bytes


Hallo el estimador plug-in del coeficiente de correlación:

In [2]:
import numpy as np

distancia_planetaria = df_distancia_nuclear['Planetary distance (AU)'].values
potencia_nuclear = df_distancia_nuclear['Nuclear power generation in Brazil (Billion kWh)'].values

# Obtengo el estimador plug-in de la correlación
estimador = np.corrcoef(distancia_planetaria, potencia_nuclear)[0, 1]

print(f'Estimador plug-in de la correlación:{estimador}')

Estimador plug-in de la correlación:0.9108140085329718


## b)
Estimar su error estándar usando bootstrap.

### Desarrollo

Defino un **n** que represente la cantidad de iteraciones bootstrap y armo una matriz de la misma longitud **n** donde voy
guardando la correlación de cada muestreo, para luego calcular su error estándar: 

In [3]:
from scipy.stats import sem

n = 1000
estimadores = np.zeros(n)
for i in range(n):
    bs_planetary_dist = np.random.choice(distancia_planetaria, size=len(distancia_planetaria), replace=True)
    bs_nuclear_pot = np.random.choice(potencia_nuclear, size=len(potencia_nuclear), replace=True)
    estimadores[i] = np.corrcoef(bs_planetary_dist, bs_nuclear_pot)[0, 1]
    
error_estandar = sem(estimadores)
print(f'Error estándar del estimador del coeficiente de correlación con bootstrap: {error_estandar}')

Error estándar del estimador del coeficiente de correlación con bootstrap: 0.005126567136792374


## c)
Encontrar un intervalo de confianza de nivel 0.95 usando los métodos normal, percentile
bootstrap, basic bootstrap y BCa.

### Desarrollo

In [4]:
from arch.bootstrap import IIDBootstrap

# Armo un intervalo de confianza 0.95 para la desviación estándar usando el método normal
intervalo_normal = (estimador - 1.96 * error_estandar, estimador + 1.96 * error_estandar)

# Armo un intervalo de confianza 0.95 para la desviación estándar usando el método percentil
intervalo_percentil = np.percentile(estimadores, [2.5, 97.5])

# Armo un intervalo de confianza 0.95 para la desviación estándar usando el basic bootstrap
intervalo_basic = (2 * estimador - np.percentile(estimadores, 97.5), 2 * estimador - np.percentile(estimadores, 2.5))

# Para armar un intervalo de confianza usando el método BCa, necesito definir la función que devuelve la correlación
# Notar que debe ser una función que reciba un solo argumento
calcular_corrocoef = lambda data: np.corrcoef(data[:, 0], data[:, 1])[0, 1]

# Armo la matriz en donde estará la muestra sobre la cual se aplica la función
muestra = np.column_stack((distancia_planetaria, potencia_nuclear))

# Ahora si uso el método bca
bs = IIDBootstrap(muestra)
intervalo_bca = bs.conf_int(calcular_corrocoef, reps=1000, method='bca')

print(f'Intervalo de confianza para la desviación estándar usando el método normal: {intervalo_normal}')
print(f'Intervalo de confianza para la desviación estándar usando el método percentil: {intervalo_percentil}')
print(f'Intervalo de confianza para la desviación estándar usando el basic bootstrap: {intervalo_basic}')
print(f'Intervalo de confianza para la desviación estándar usando el BCa bootstrap: {intervalo_bca[0]} , {intervalo_bca[1]}')

Intervalo de confianza para la desviación estándar usando el método normal: (np.float64(0.9007659369448587), np.float64(0.9208620801210848))
Intervalo de confianza para la desviación estándar usando el método percentil: [-0.29938741  0.33635065]
Intervalo de confianza para la desviación estándar usando el basic bootstrap: (np.float64(1.4852773656916438), np.float64(2.121015430134203))
Intervalo de confianza para la desviación estándar usando el BCa bootstrap: [0.8583493] , [0.93882955]
